In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install selenium
!pip install webdriver_manager
!pip3 install selenium-wire
!pip install undetected_chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and plat

In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.h6gOSZBVP5/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.OruZwXTyeP/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.65KLESnagM/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:12 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Hit:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:14 http://security.ubuntu.com/ubuntu jammy-securit

In [ ]:
#import libraries
import time
import pandas as pd
import numpy as np
import requests
import math
import glob
import os
import re
from dateutil import parser
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select
from selenium.webdriver import Firefox, Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import seleniumwire.undetected_chromedriver as uc
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains

def load_soup(URL):
    # Function to load a webpage and parse its content with BeautifulSoup
    page_content = requests.get(URL, timeout=None, headers={
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.94 Safari/537.36"
    }).content

    soup = BeautifulSoup(page_content, 'html.parser')
    return soup

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-blink-features=AutomationControlled')

    prefs = {
        "profile.default_content_setting_values.geolocation": 2  # 1 for allow, 2 for block
    }
    options.add_experimental_option("prefs", prefs)

    #options.add_argument("--headless")
    #options.page_load_strategy = 'eager'
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--start-maximized")
    #options.add_experimental_option("desired_capabilities", capa)
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
driver = web_driver()
wait = WebDriverWait(driver, 30)
data = []

In [ ]:
driver.maximize_window()

In [ ]:
url = "https://app.hyrecar.com/offers?address=Chicago%2C%20IL%2C%20USA&addressComponents=%5Bobject%20Object%5D&addressComponents=%5Bobject%20Object%5D&addressComponents=%5Bobject%20Object%5D&addressComponents=%5Bobject%20Object%5D&bounds=%7B%22nw%22%3A%7B%22lat%22%3A42.35079450612662%2C%22lng%22%3A-87.9758675359375%7D%2C%22se%22%3A%7B%22lat%22%3A41.40191049200533%2C%22lng%22%3A-87.2837288640625%7D%2C%22sw%22%3A%7B%22lat%22%3A41.40191049200533%2C%22lng%22%3A-87.9758675359375%7D%2C%22ne%22%3A%7B%22lat%22%3A42.35079450612662%2C%22lng%22%3A-87.2837288640625%7D%7D&city=Chicago&daily_from=NaN&daily_to=NaN&distance=33&end_date=0000-00-00%2000%3A00%3A00&geoApproved=false&geo_lat=41.8781136&geo_lng=-87.6297982&getAddressComponent=function%20%28e%2Ct%29%7Breturn%20e.find%28%28function%28e%29%7Breturn%20e.types.includes%28t%29%7D%29%29%7C%7C%7B%7D%7D&limit=20&loading=false&map=false&page=1&sortBy=Relevance&start_date=0000-00-00%2000%3A00%3A00&state=IL&zip=60604"

driver.get(url)

In [ ]:
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                       "input[name='username']")));

driver.execute_script("window.stop();")

In [ ]:
driver.find_element(By.CSS_SELECTOR, "input[name='username']").send_keys("YOUR-USERNAME")

In [ ]:
driver.find_element(By.CSS_SELECTOR, "input[name='password']").send_keys("YOUR-PASSWORD")

In [ ]:
login_button = driver.find_elements(By.CSS_SELECTOR, "button.MuiButton-root")[3]

In [ ]:
driver.execute_script("arguments[0].click();", login_button)

In [ ]:
wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,
                                       "div.sc-bwzfXH.ihgQcP.sc-htpNat.jtWIOA > div > div")));

driver.execute_script("window.stop();")
time.sleep(10)

In [ ]:
number_of_pages = int(driver.find_element(By.CSS_SELECTOR, "ul.pager > p").text.split(" ")[-1])

In [ ]:
print(number_of_pages)

3


In [ ]:
dmc = []

pg = 1
while True:
    print(pg)
    pg+=1

    cards = driver.find_elements(By.CSS_SELECTOR,
                                 "div.sc-bwzfXH.ihgQcP.sc-htpNat.jtWIOA > div > div")

    print(f"Len of card is {len(cards)}")
    for i in cards:
        try:
            container = i.find_element(By.CSS_SELECTOR, "div")
        except:
            container = i.find_element(By.CSS_SELECTOR, "div")


        try:
            dmc.append(container.get_attribute("data-metrics-carid"))
        except:
            pass


    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "ul.pager > li.next > a")

        driver.execute_script("arguments[0].click();", next_button)
        wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,
                                       "div.sc-bwzfXH.ihgQcP.sc-htpNat.jtWIOA > div > div")));
        # Scroll to the bottom of the page using JavaScript
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(7)
    except:
        break

1
Len of card is 20
2
Len of card is 20
3
Len of card is 20


In [ ]:
len(dmc)

60

In [ ]:
len(set(dmc))

60

In [ ]:
car_links = ["https://app.hyrecar.com/offers/" + i for i in dmc]

In [ ]:
print(len(car_links))

60


In [ ]:
def get_reviews_count(ele):
  for data in ele.find_all("h4"):
    if "Reviews" in data.get_text():
      no_of_review = re.sub("[^0-9]", "", data.text)

  return no_of_review

def get_insurance_fee(ele):

  for info in ele.find_all("td", {'class': "sc-jnlKLf kPYPuu"}):
    if "Insurance Fee" in info.get_text():
      match = re.search(r'\$\d+\.\d+', info.get_text())

      if match:
        number_str = match.group()
        number = float(number_str.replace('$', ''))

  return number

def get_discount(driver):
  days = []
  percentage_values = []


  actions = ActionChains(driver)

  # Retrieve WebElement
  element = driver.find_element(By.XPATH, "//button[normalize-space()='Want a discount?']")

  # Use ActionChains class to mouse hover
  actions.move_to_element(element).perform()

  # Retrieve the tooltip element
  tool_tip = driver.find_element(By.CSS_SELECTOR, ".tooltip-inner")

  # Get the tooltip text
  tool_tip_text = tool_tip.text

  days = re.findall(r'Book for (\d+) days', tool_tip_text)

  # Convert the extracted days to integers
  days = [int(day) for day in days]

  # Regular expression to find percentages
  percentages = re.findall(r'\d+%', tool_tip_text)

  # Convert percentages to integers (without the % symbol)
  percentage_values = [int(p.strip('%')) for p in percentages]

  return days, percentage_values


In [ ]:
table = []
no = 1
for link in car_links:
  data = {}


  try:
    driver.get(link)
    element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//table[@class='table table-striped table-condensed']")))
    soup = BeautifulSoup(driver.page_source)
    print(no)
    no = no+1

    #extract name
    try:
      name = ""
      for tag in soup.find("div", {'class' : "sc-bwzfXH sc-jqCOkK kzlbmn"}).find_all("h5"):
        name = name + tag.text + " "

      data['Name'] = name.strip(" ")

    except:
      data['Name'] = ""

    #extract mileage(mi/day)
    try:
      mileage = re.sub("[A-za-z/ ]", "", soup.find("div", {'class' : "sc-bwzfXH ihgQcP"}).text)
      data["Mileage (mi/day)"] = int(mileage)

    except:
      data["Mileage (mi/day)"] = np.nan

    #extracting car owners
    try:
      car_owner = soup.find("div", {'class' : "sc-bwzfXH sc-frDJqD cZviTZ"}).find("h3").text
      data["Car Owners"] = car_owner.strip(" ")

    except:
      data["Car Owners"] = ""

    #extracting price($)
    try:
      price = re.sub("[$]", "", soup.find("div", {'class' : "sc-bwzfXH sc-uJMKN dBiyme"}).find("h3").text)
      data["Prices ($)"] = int(price)

    except:
      data["Prices Per Day ($)"] = np.nan

    try:
      data["No of Reviews"] = get_reviews_count(soup)

    except:
      data["No of Reviews"] = ""

    try:
      data["Insurance Fee"] = get_insurance_fee(soup)

    except:
      data["Insurance Fee"] = np.nan

    #extracting image url
    try:
      image_url = soup.find("img", {'class' : "embed-responsive-item"}).get('src')
      data["Image URL"] = image_url

    except:
      data["Image URL"] = ""

    try:
      description = soup.find("p", {'class' : "sc-kAzzGY ejEbWn"}).text
      data["Description"] = description

    except:
      data["Description"] = ""

    try:
      days, percentage_values = get_discount(driver)

      for n in range(len(days)):
        data[f"{days[n]} Days discount" ] = percentage_values[n]

    except:
      pass


  except:
    pass

  table.append(data)

  # time.sleep(10)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [ ]:
df = pd.DataFrame(table)
df.head()

,Name,Mileage (mi/day),Car Owners,Prices ($),No of Reviews,Insurance Fee,Image URL,Description,7 Days discount,30 Days discount
0,2015 Jetta Volkswagen,500,CHRIS AUTO,40,2,19.71,https://static.hyrecar.com/https://s3.amazonaw...,Runs and drives great \nReady for rideshare an...,NaN,NaN
1,2012 Fit Honda,1000,House of,45,0,20.31,https://static.hyrecar.com/https://s3.amazonaw...,For delivery and ride share,NaN,NaN
2,2015 Camry Toyota,1000,braulio,59,4,21.99,https://d1faaofv0bplxp.cloudfront.net/car_plac...,Very nice looking car\nGas saver \nBluetooth r...,5.0,10.0
3,2019 Fusion Ford,250,Paul,50,0,20.91,https://static.hyrecar.com/https://s3.amazonaw...,"*** Delivery Only ***\n\nVery nice and clean,,...",5.0,10.0
4,2013 RDX Acura,1000,Balaji,42,0,19.95,https://static.hyrecar.com/https://s3.amazonaw...,*** Delivery Only ***\n\nThe car is in excelle...,10.0,15.0


In [ ]:
import datetime

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as a string
current_time = now.strftime("%d-%m-%Y")

# Determine the time of day
if 5 <= now.hour < 12:
    time_of_day = "morning"
elif 12 <= now.hour < 18:
    time_of_day = "afternoon"
else:
    time_of_day = "evening"

df.to_csv(f'/content/drive/MyDrive/Work/project ppt/hyrecar/hyrecar_{current_time}_{time_of_day}.csv')

In [ ]:
driver.quit()